In [2]:
import requests
import numpy as np
import time
import pandas as pd
import os

In [3]:
API_KEY = os.environ.get("RIOT_API_KEY")

In [18]:
TEST_SUMMONER_ID = 'n3o_NHWmIpmHHknm468Hs481qOJmIiadIXneS76JDtsXYF5e'

In [4]:
def get_summoner_ids(region: str, num_pages=4):
    """
    Gets random summoner ids from North America from the various ranks (Diamond:Platinum:Gold:Silver:Bronze)
    Returns a dictionary by rank of summoner ids
    """
    ranks = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE']
    tiers = ['I', 'II', 'III', 'IV']
    summoner_ids_rank = dict()
    for rank in ranks:
        summoner_ids = set()
        for tier in tiers:
            for page_num in range(1, num_pages+1):
                URL = f'https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{tier}/?page={page_num}&api_key={API_KEY}'
                response = requests.get(URL)
                summoner_ids.update([s['summonerId'] for s in response.json()])
                time.sleep(1.2)
        summoner_ids_rank[rank] = summoner_ids
    return summoner_ids_rank

In [5]:
URL = f'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/PLATINUM/I/?page=1&api_key={API_KEY}'
response = requests.get(URL)
test_summoner_id = response.json()[0]['summonerId']

In [7]:
test_summoner_id

'n3o_NHWmIpmHHknm468Hs481qOJmIiadIXneS76JDtsXYF5e'

In [6]:
na1_summoner_ids = get_summoner_ids('na1')

In [4]:
euw1_summoner_ids = get_summoner_ids('euw1')

In [5]:
kr_summoner_ids = get_summoner_ids('kr')

In [7]:
def champion_lane_assignments():
    """
    Champion lane assignments according to op.gg on 4/21/2022
    """
    top_champions = ['Aatrox', 'Akali', "chogath", 
                     'Camille', 'Darius', 'DrMundo', 
                     'Fiora', 'gangplank', 'Garen', 'Gnar', 
                     'Gragas', 'Gwen', 'Heimerdinger', 'Illaoi', 
                     'Irelia', 'Jax', 'Jayce', 'Kayle', 'Kennen', 
                     'Kled', 'Malphite', 'Mordekaiser', 'Nasus', 
                     'Ornn', 'Quinn', 'Renekton', 'Rengar', 'Riven', 
                     'Rumble', 'Sett', 'Shen', 'Singed', 'Sion', 
                     'TahmKench', 'Teemo', 'Tryndamere', 'Urgot', 'Volibear', 'Yorick']
    jungle_champions = ['Diana', 'Ekko', 'Elise', 'Evelynn', 'Fiddlesticks',
                        'Graves', 'Hecarim', 'Ivern', 'JarvanIV', 'Karthus',
                        'Kayn', "KhaZix", 'Kindred', 'LeeSin', 'Lillia',
                        'MasterYi', 'Nidalee', 'Nocturne', 'nunu', 'Olaf',
                        'Poppy', 'Rammus', "RekSai", 'Sejuani', 'Shaco',
                        'Shyvana', 'Skarner', 'Taliyah', 'Talon', 'Trundle',
                        'Udyr', 'Vi', 'Viego', 'Warwick', 'monkeyking', 'XinZhao', 'Zac']
    middle_champions = ['Ahri', 'Akshan', 'Anivia', 'Annie', 'aurelionsol', 
                        'Azir', 'Cassiopeia', 'Corki', 'Fizz', 'Galio', 'Kassadin',
                        'Katarina', 'LeBlanc', 'Lissandra', 'Malzahar', 'Neeko', 'Orianna',
                        'Qiyana', 'Ryze', 'Sylas', 'Syndra', 'TwistedFate', 'Veigar',
                        'Vex', 'Viktor', 'Vladimir', 'Yasuo', 'Yone', 'Zed', 'Zoe']
    bot_champions = ['Aphelios', 'Ashe', 'Caitlyn', 'Draven', 'Ezreal', 'Jhin', 'Jinx',
                     "kaisa", 'Kalista', "kogmaw", 'Lucian', 'MissFortune', 'Samira',
                     'Sivir', 'Tristana', 'Twitch', 'Varus', 'Vayne', 'Xayah',
                     'Zeri', 'Ziggs']
    support_champions = ['Alistar', 'Amumu', 'Bard', 'Blitzcrank', 'Brand', 'Braum', 'Janna',
                         'Karma', 'Leona', 'Lulu', 'Lux', 'Maokai', 'Morgana', 'Nami',
                         'Nautilus', 'Pantheon', 'Pyke', 'Rakan', 'Rell', 
                         'renata', 'Senna', 'Seraphine',
                         'Sona', 'Soraka', 'Swain', 'Taric', 'Thresh', 
                         "velkoz", 'Xerath', 'Yuumi', 'Zilean', 'Zyra']
    top_champions = set([x.lower() for x in top_champions])
    jungle_champions = set([x.lower() for x in jungle_champions])
    middle_champions = set([x.lower() for x in middle_champions])
    bot_champions = set([x.lower() for x in bot_champions])
    support_champions = set([x.lower() for x in support_champions])
    return {'top': top_champions, 'mid': middle_champions, 'jg': jungle_champions, 
            'bot': bot_champions, 'sup': support_champions}

In [8]:
import json

In [11]:
with open('data/champion_lane_mapping.txt', 'w') as f: 
    lane_assigments_dict = champion_lane_assignments()
    f.write(json.dumps({k:list(lane_assigments_dict[k]) for k in lane_assigments_dict}))

In [25]:
def get_champion_masteries(summoner_id_rank: dict, region: str):
    """
    Gets the champion mastery information for each summoner id by rank in the given regio
    
    Returns a dictionary by rank containing the list of each players top 5 champions
    """
    min_champs = 5
    min_exp = 21000
    
    print(region)
    champion_masteries = dict()
    
    for rank in summoner_id_rank:
        champions_by_rank = []
        for _, summoner_id in enumerate(summoner_id_rank[rank]):
            URL = f'https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{summoner_id}?api_key={API_KEY}'
            response = requests.get(URL)
            champion_list = response.json()
            
            if len(champion_list) < min_champs:
                continue
            
            if champion_list[4]['championPoints'] < min_exp:
                continue
            
            champs = []
            for i in range(5):
                attributes = (champion_list[i]['championId'], champion_list[i]['championPoints'])
                champs.extend(attributes)
            champions_by_rank.append(champs)
            time.sleep(1.2)
            if _ % 50 == 0:
                print(f"finished summoner id at index {_}")
        champion_masteries[rank] = champions_by_rank
        print(f'Finished processing rank: {rank}')
        time.sleep(60)
    return champion_masteries

In [9]:
import json
raw_champions = None
with open('data/champions.json', encoding="cp866") as f:
    raw_champions = json.load(f)

In [10]:
champion_ids_name = {}
for champ in raw_champions['data'].keys():
    champion_ids_name[champ.lower()] = int(raw_champions['data'][champ]['key'])

In [11]:
lane_assigments = champion_lane_assignments()

In [12]:
lane_assigments_ids = dict()
for lane in lane_assigments:
    ids_for_lane = []
    for champ in lane_assigments[lane]:
        ids_for_lane.append(champion_ids_name[champ])
    lane_assigments_ids[lane] = set(ids_for_lane)

In [11]:
champion_ids_name.keys()

dict_keys(['aatrox', 'ahri', 'akali', 'akshan', 'alistar', 'amumu', 'anivia', 'annie', 'aphelios', 'ashe', 'aurelionsol', 'azir', 'bard', 'blitzcrank', 'brand', 'braum', 'caitlyn', 'camille', 'cassiopeia', 'chogath', 'corki', 'darius', 'diana', 'draven', 'drmundo', 'ekko', 'elise', 'evelynn', 'ezreal', 'fiddlesticks', 'fiora', 'fizz', 'galio', 'gangplank', 'garen', 'gnar', 'gragas', 'graves', 'gwen', 'hecarim', 'heimerdinger', 'illaoi', 'irelia', 'ivern', 'janna', 'jarvaniv', 'jax', 'jayce', 'jhin', 'jinx', 'kaisa', 'kalista', 'karma', 'karthus', 'kassadin', 'katarina', 'kayle', 'kayn', 'kennen', 'khazix', 'kindred', 'kled', 'kogmaw', 'leblanc', 'leesin', 'leona', 'lillia', 'lissandra', 'lucian', 'lulu', 'lux', 'malphite', 'malzahar', 'maokai', 'masteryi', 'missfortune', 'monkeyking', 'mordekaiser', 'morgana', 'nami', 'nasus', 'nautilus', 'neeko', 'nidalee', 'nocturne', 'nunu', 'olaf', 'orianna', 'ornn', 'pantheon', 'poppy', 'pyke', 'qiyana', 'quinn', 'rakan', 'rammus', 'reksai', 'rell

In [24]:
for i in lane_assigments:
    for j in range(len(lane_assigments[i])):
        if lane_assigments[i][j] not in champion_ids_name.keys():
            print(lane_assigments[i][j])

TypeError: 'set' object is not subscriptable

In [74]:
sum({lane:1 for lane in lane_assigments}.values())

5

In [13]:
def get_top3_by_lane(champion_list):
    """
    Returns the player's top 3 champions by lane.
    """
    result = []
    num_counted = {lane:0 for lane in lane_assigments_ids}
    for i in range(len(champion_list)):
        for lane in lane_assigments_ids:
            if champion_list[i]['championId'] in lane_assigments_ids[lane] and num_counted[lane] < 3:
                num_counted[lane] += 1
                attributes = (champion_list[i]['championId'], champion_list[i]['championPoints'], lane)
                result.extend(attributes)
        if sum(num_counted.values()) >= 15:
            return result
    

In [14]:
def get_champion_top3_by_lane(summoner_id_rank: dict, region: str):
    """
    Gets the champion mastery information for each summoner id by rank.
    
    Returns a dictionary by rank containing the list of each players top 3 champions per lane.
    """
    min_champs = 16
    min_exp = 10_000
    
    print(region)
    champion_masteries = dict()
    
    for rank in summoner_id_rank:
        champions_by_rank = []
        for _, summoner_id in enumerate(summoner_id_rank[rank]):
            URL = f'https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{summoner_id}?api_key={API_KEY}'
            response = requests.get(URL)
            champion_list = response.json()
            
            if len(champion_list) < min_champs:
                continue
            
            if champion_list[min_champs-1]['championPoints'] < min_exp:
                continue
            
            top_3 = get_top3_by_lane(champion_list)
            if top_3 != None:
                champions_by_rank.append(top_3)
            time.sleep(1.2)
            if _ % 50 == 0:
                print(f"finished summoner id at index {_}")
        champion_masteries[rank] = champions_by_rank
        print(f'Finished processing rank: {rank}')
        time.sleep(60)
    return champion_masteries

In [18]:
URL = f'https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{TEST_SUMMONER_ID}?api_key={API_KEY}'
response = requests.get(URL)
champion_list = response.json()

NameError: name 'TEST_SUMMONER_ID' is not defined

In [14]:
champion_list[0]

{'championId': 86,
 'championLevel': 7,
 'championPoints': 203010,
 'lastPlayTime': 1648790653000,
 'championPointsSinceLastLevel': 181410,
 'championPointsUntilNextLevel': 0,
 'chestGranted': True,
 'tokensEarned': 0,
 'summonerId': 'n3o_NHWmIpmHHknm468Hs481qOJmIiadIXneS76JDtsXYF5e'}

In [15]:
na1_top3_by_lane = get_champion_top3_by_lane(na1_summoner_ids, 'na1')

na1
finished summoner id at index 0
finished summoner id at index 100
finished summoner id at index 400
finished summoner id at index 450
finished summoner id at index 1550
finished summoner id at index 1800
finished summoner id at index 2200
finished summoner id at index 2500
Finished processing rank: DIAMOND
finished summoner id at index 0
finished summoner id at index 50
finished summoner id at index 300
finished summoner id at index 350
finished summoner id at index 600
finished summoner id at index 900
finished summoner id at index 950
finished summoner id at index 1100
finished summoner id at index 1150
finished summoner id at index 1400
finished summoner id at index 1750
finished summoner id at index 1950
finished summoner id at index 2000
finished summoner id at index 2300
finished summoner id at index 2600
finished summoner id at index 3000
finished summoner id at index 3050
Finished processing rank: PLATINUM
finished summoner id at index 0
finished summoner id at index 250
fi

In [16]:
len(na1_top3_by_lane['BRONZE'])

484

In [9]:
na1_top5_champs = get_champion_masteries(na1_summoner_ids, 'na1')

na1
Finished processing rank: DIAMOND
Finished processing rank: PLATINUM
Finished processing rank: GOLD
Finished processing rank: SILVER
Finished processing rank: BRONZE


In [128]:
na1_top5_champs['DIAMOND'][1]

[84, 147, 157, 235, 62]

In [18]:
columns_top3_by_lane = []
for i in range(15):
    columns_top3_by_lane.extend(f'Champion{i+1} Mastery{i+1} Lane{i+1}'.split())
columns_top3_by_lane[:3]

['Champion1', 'Mastery1', 'Lane1']

In [58]:
for rank in na1_top3_by_lane:
    for i, l in enumerate(na1_top3_by_lane[rank]):
        if l == None:
            na1_top3_by_lane[rank].pop(i)

In [65]:
na1_top3_diamond = pd.DataFrame(na1_top3_by_lane['DIAMOND'], columns=columns_top3_by_lane)
na1_top3_diamond

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,51,272295,bot,145,88254,bot,103,74100,mid,84,...,jg,99,9746,sup,254,7431,jg,114,1286,top
1,55,67222,mid,238,51605,mid,3,51363,mid,39,...,sup,16,10862,sup,202,9299,bot,21,8998,bot
2,39,956886,top,90,109817,mid,98,74811,top,245,...,mid,350,16135,sup,1,14686,mid,18,13146,bot
3,64,154967,jg,517,97859,mid,238,66095,mid,104,...,bot,145,17146,bot,497,6812,sup,555,6105,sup
4,131,514349,jg,103,214265,mid,99,163929,sup,25,...,mid,38,10785,mid,11,9387,jg,113,6642,jg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,84,161127,top,55,52375,mid,13,50879,mid,887,...,bot,235,16407,sup,147,15067,sup,64,10242,jg
442,122,517155,top,875,97869,top,11,94504,jg,266,...,bot,81,7643,bot,53,4131,sup,555,3336,sup
443,98,1172416,top,24,271685,top,157,76715,mid,84,...,jg,56,10990,jg,119,7660,bot,145,7129,bot
444,145,301326,bot,29,192565,bot,245,150420,jg,8,...,top,412,44736,sup,555,44171,sup,84,44148,top


In [66]:
for rank in na1_top3_by_lane:
    na_1_new_df = pd.DataFrame(na1_top3_by_lane[rank], columns=columns_top3_by_lane)
    na_1_new_df.to_csv(f'data/top3/na_{rank.lower()}_top3_champ_mastery_lan.csv', index=False)

In [21]:
columns_top5 = ['Champion1', 'Mastery1', 'Champion2', 'Mastery2', 'Champion3', 'Mastery3', 'Champion4', 'Mastery4', 'Champion5', 'Mastery5']

In [130]:
na_1_diamond = pd.DataFrame(na1_top5_champs['DIAMOND'], columns=columns_top5)

In [131]:
na_1_diamond.head()

,Champion1,Champion2,Champion3,Champion4,Champion5
0,145,8,555,81,67
1,84,147,157,235,62
2,76,64,104,121,131
3,432,25,38,30,79
4,29,555,350,235,267


In [22]:
for rank in na1_top5_champs:
    na_1_new_df = pd.DataFrame(na1_top5_champs[rank], columns=columns_top5)
    na_1_new_df.to_csv(f'data/na_{rank.lower()}_top5_champ_masteries.csv', index=False)

In [10]:
euw1_top5_champs = get_champion_masteries(euw1_summoner_ids, 'euw1')

euw1
Finished processing rank: DIAMOND
Finished processing rank: PLATINUM
Finished processing rank: GOLD
Finished processing rank: SILVER
Finished processing rank: BRONZE


In [15]:
euw_top3_by_lane = get_champion_top3_by_lane(euw1_summoner_ids, 'euw1')

euw1
finished summoner id at index 100
finished summoner id at index 250
finished summoner id at index 300
finished summoner id at index 400
finished summoner id at index 450
finished summoner id at index 500
finished summoner id at index 550
finished summoner id at index 600
finished summoner id at index 650
finished summoner id at index 750
finished summoner id at index 900
finished summoner id at index 950
finished summoner id at index 1000
finished summoner id at index 1050
finished summoner id at index 1150
finished summoner id at index 1200
finished summoner id at index 1250
finished summoner id at index 1300
finished summoner id at index 1350
finished summoner id at index 1400
finished summoner id at index 1450
finished summoner id at index 1550
finished summoner id at index 1600
finished summoner id at index 1650
finished summoner id at index 1700
finished summoner id at index 1750
finished summoner id at index 1800
finished summoner id at index 1850
finished summoner id at ind

In [19]:
for rank in euw_top3_by_lane:
    na_1_new_df = pd.DataFrame(euw_top3_by_lane[rank], columns=columns_top3_by_lane)
    na_1_new_df.to_csv(f'data/top3/euw_{rank.lower()}_top3_champ_mastery_lan.csv', index=False)

In [23]:
for rank in euw1_top5_champs:
    euw_new_df = pd.DataFrame(euw1_top5_champs[rank], columns=columns)
    euw_new_df.to_csv(f'data/euw_{rank.lower()}_top5_champ_masteries.csv', index=False)

In [8]:
len(kr_summoner_ids['BRONZE'])

3220

In [ ]:
kr_summoner_ids

In [26]:
kr_top5_champs = get_champion_masteries(kr_summoner_ids, 'kr')

kr
finished summoner id at index 0
finished summoner id at index 50
finished summoner id at index 100
finished summoner id at index 200
finished summoner id at index 250
finished summoner id at index 300
finished summoner id at index 350
finished summoner id at index 400
finished summoner id at index 450
finished summoner id at index 600
finished summoner id at index 650
finished summoner id at index 700
finished summoner id at index 750
finished summoner id at index 800
finished summoner id at index 900
finished summoner id at index 950
finished summoner id at index 1000
finished summoner id at index 1100
finished summoner id at index 1150
finished summoner id at index 1200
finished summoner id at index 1250
finished summoner id at index 1300
finished summoner id at index 1350
finished summoner id at index 1400
finished summoner id at index 1450
finished summoner id at index 1550
finished summoner id at index 1650
finished summoner id at index 1700
finished summoner id at index 1750
f

In [20]:
kr_top3_by_lane = get_champion_top3_by_lane(kr_summoner_ids, 'kr')

kr
finished summoner id at index 50
finished summoner id at index 100
finished summoner id at index 200
finished summoner id at index 250
finished summoner id at index 600
finished summoner id at index 750
finished summoner id at index 800
finished summoner id at index 1000
finished summoner id at index 1050
finished summoner id at index 1200
finished summoner id at index 1250
finished summoner id at index 1350
finished summoner id at index 1400
finished summoner id at index 1450
finished summoner id at index 1500
finished summoner id at index 1550
finished summoner id at index 1650
finished summoner id at index 1700
finished summoner id at index 1750
finished summoner id at index 1800
finished summoner id at index 1900
finished summoner id at index 2000
finished summoner id at index 2100
finished summoner id at index 2150
finished summoner id at index 2200
finished summoner id at index 2250
finished summoner id at index 2400
finished summoner id at index 2450
finished summoner id at i

In [27]:
for rank in kr_top5_champs:
    kr_new_df = pd.DataFrame(kr_top5_champs[rank], columns=columns)
    kr_new_df.to_csv(f'data/kr_{rank.lower()}_top5_champ_masteries.csv', index=False)

In [ ]:
for rank in kr_top3_by_lane:
    na_1_new_df = pd.DataFrame(kr_top3_by_lane[rank], columns=columns_top3_by_lane)
    na_1_new_df.to_csv(f'data/top3/kr_{rank.lower()}_top3_champ_mastery_lane.csv', index=False)